In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt

from pymedphys.level1.mudensity import *

DATA_DIRECTORY = "../../tests/data"
DELIVERY_DATA_FILEPATH = os.path.join(DATA_DIRECTORY, 'mu_density_example_arrays.npz')
regress_test_arrays = np.load(DELIVERY_DATA_FILEPATH)

In [ ]:
mu = regress_test_arrays['mu']
mlc = regress_test_arrays['mlc']
jaw = regress_test_arrays['jaw']

leaf_pair_widths = np.array(AGILITY_LEAF_PAIR_WIDTHS)
max_leaf_gap=400
grid_resolution=2.5
time_steps=50

mu, mlc, jaw = remove_irrelevant_control_points(mu, mlc, jaw)

In [ ]:
def determine_full_grid(max_leaf_gap, grid_resolution, leaf_pair_widths):
    leaf_x = np.arange(
        -max_leaf_gap/2,
        max_leaf_gap/2 + grid_resolution,
        grid_resolution).astype('float')

    _, initial_leaf_grid_y_pos = determine_leaf_y(leaf_pair_widths)

    total_leaf_widths = np.sum(leaf_pair_widths)
    top_grid_pos = (
        np.ceil((total_leaf_widths/2 - initial_leaf_grid_y_pos) / grid_resolution) *
        grid_resolution + initial_leaf_grid_y_pos)

    bot_grid_pos = (
        initial_leaf_grid_y_pos -
        np.ceil((total_leaf_widths/2 + initial_leaf_grid_y_pos) / grid_resolution) *
        grid_resolution)

    grid_y = np.arange(
        bot_grid_pos, top_grid_pos + grid_resolution, grid_resolution)

    grid_xx, grid_yy = np.meshgrid(leaf_x, grid_y)

    return grid_xx, grid_yy

In [ ]:
def determine_calc_grid_and_adjustments(mlc, jaw, leaf_pair_widths,
                                        grid_resolution):
    min_y = np.min(-jaw[:, 0])
    max_y = np.max(jaw[:, 1])

    leaf_y, initial_leaf_grid_y_pos = determine_leaf_y(leaf_pair_widths)

    top_grid_pos = (
        np.ceil((max_y - initial_leaf_grid_y_pos) / grid_resolution) *
        grid_resolution + initial_leaf_grid_y_pos)

    bot_grid_pos = (
        initial_leaf_grid_y_pos -
        np.ceil((-min_y + initial_leaf_grid_y_pos) / grid_resolution) *
        grid_resolution)

    grid = dict()
    grid['jaw'] = np.arange(
        bot_grid_pos, top_grid_pos + grid_resolution, grid_resolution
    ).astype('float')

    grid_leaf_map = np.argmin(
        np.abs(grid['jaw'][:, None] - leaf_y[None, :]), axis=1)

    adjusted_grid_leaf_map = grid_leaf_map - np.min(grid_leaf_map)

    leaves_to_be_calced = np.unique(grid_leaf_map)
    adjusted_mlc = mlc[:, leaves_to_be_calced, :]

    min_x = np.floor(
        np.min(-adjusted_mlc[:, :, 0]) / grid_resolution) * grid_resolution
    max_x = np.ceil(
        np.max(adjusted_mlc[:, :, 1]) / grid_resolution) * grid_resolution

    grid['mlc'] = np.arange(
        min_x, max_x + grid_resolution, grid_resolution
    ).astype('float')

    return grid, adjusted_grid_leaf_map, adjusted_mlc

In [ ]:
i = 3
control_point_slice = slice(i, i + 2, 1)

current_mlc = mlc[control_point_slice, :, :]
# current_jaw = jaw[control_point_slice, :]

current_jaw = np.array([
    [2, 1],
    [16, -4]
])

delivered_mu = np.diff(mu[control_point_slice])

(
    grid, grid_leaf_map, current_mlc
) = determine_calc_grid_and_adjustments(
    current_mlc, current_jaw, 
    leaf_pair_widths, grid_resolution)

In [ ]:
current_mlc

In [ ]:
current_jaw

In [ ]:
delivered_mu

In [ ]:
grid

In [ ]:
grid_leaf_map

In [ ]:
positions = {
    'mlc': {
        1: (-current_mlc[0, :, 0], -current_mlc[1, :, 0]), # left
        -1: (current_mlc[0, :, 1], current_mlc[1, :, 1]) # right
    },
    'jaw': {
        1: (-current_jaw[0::-1, 0], -current_jaw[1::, 0]), # bot
        -1: (current_jaw[0::-1, 1], current_jaw[1::, 1]) # top
    }
}

positions

In [ ]:
def calc_blocked_t(travel_diff, grid_resolution):
    blocked_t = np.ones_like(travel_diff) * np.nan

    fully_blocked = travel_diff <= -grid_resolution/2
    fully_open = travel_diff >= grid_resolution/2
    blocked_t[fully_blocked] = 1
    blocked_t[fully_open] = 0

    transient = ~fully_blocked & ~fully_open

    blocked_t[transient] = (
        (-travel_diff[transient] + grid_resolution/2) / 
        grid_resolution)

    assert np.all(~np.isnan(blocked_t))
    
    return blocked_t

In [ ]:
multiplier = -1
device = 'mlc'
start, end = positions[device][multiplier]


dt = (end - start) / (time_steps - 1)
travel = start[None, :] + np.arange(0,time_steps)[:, None] * dt[None, :]

np.shape(travel)
np.shape(multiplier * (grid[device][None,None, :] - travel[:, :, None]))

In [ ]:
blocked_by_device = {}

for device, value in positions.items():
    blocked_by_device[device] = dict()
    
    for multiplier, (start, end) in value.items():
        dt = (end - start) / 49
        travel = start[None, :] + np.arange(0,time_steps)[:, None] * dt[None, :]
        travel_diff = multiplier * (grid[device][None, None, :] - travel[:, :, None])
        
        blocked_by_device[device][multiplier] = calc_blocked_t(
            travel_diff, grid_resolution)

In [ ]:
device_open = {}

for device, value in blocked_by_device.items():
    device_sum = np.sum(np.concatenate([
        np.expand_dims(blocked, axis=0)
        for _, blocked in value.items()
    ], axis=0), axis=0)
    
    device_open[device] = 1 - device_sum


In [ ]:
np.shape(device_open['mlc'])

In [ ]:
np.shape(device_open['jaw'])

In [ ]:
mlc_open = device_open['mlc'][:, grid_leaf_map, :]
jaw_open = device_open['jaw'][:, 0, :]

In [ ]:
np.shape(mlc_open)

In [ ]:
np.shape(jaw_open)

In [ ]:
open_t = mlc_open * jaw_open[:, :, None]

In [ ]:
open_fraction = np.mean(open_t, axis=0)
np.shape(open_fraction)

In [ ]:
grid['mlc']

In [ ]:
grid['jaw']

In [ ]:
plt.pcolormesh(grid['mlc'], grid['jaw'], open_fraction)
plt.colorbar()
plt.title('MU density')
plt.xlabel('MLC direction (mm)')
plt.ylabel('Jaw direction (mm)')
plt.axis('equal')
plt.gca().invert_yaxis()

In [ ]:
current_mlc

In [ ]:
current_jaw